BDLE 2023

date du document  :  10/11/2023

# TP7 2023 Structured Streaming  (ETUDIANTS)
#### BORCHANI Sofia 21212080
#### BOUCHOUCHI Nour 28600211
#### FAURE Guillaume 21108390
#### ZIDAT Lydia 21212792





## Préparation du TP

Installer pyspark et findspark (durée 1'):


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Démarrer la session spark

In [ ]:
import os
import glob

pyspark_dir = glob.glob('/usr/local/lib/python*/dist-packages/pyspark')[0]
print("pyspark directory is", pyspark_dir)
os.environ["SPARK_HOME"] = pyspark_dir
os.environ["JAVA_HOME"] = "/usr"

pyspark directory is /usr/local/lib/python3.10/dist-packages/pyspark


In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# pour les dataframe et udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "4G").\
  set("spark.driver.memory","4G").\
  set("spark.sql.catalogImplementation","in-memory")

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

# pour le TP streaming : 1 seule partition pour avoir un seul fichier de sortie par exécution
spark.conf.set("spark.sql.shuffle.partitions", "1")

session démarrée, son id est  local-1700241776111


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select')or r.lower().startswith('with'))
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [ ]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


## Exercice 1 : group by window

On commence par étudier le cas de données statiques.

On considère la table suivante : S1(time, v)

In [ ]:
data1 = """
8:06 A
8:10 B
8:12 B
8:12 B
8:12 A
8:20 C
8:20 B
8:20 B
"""
date_prefix = "2023-11-10T"

tab = [line.split(" ") for line in data1.split('\n') if len(line)>0]
tab = [(date_prefix+n, v) for (n, v) in tab]
S1 = spark.createDataFrame(tab, ["time", "v"]).selectExpr("timestamp(time) as time", "v")
S1.printSchema()
display(S1.selectExpr("date_format(time, 'HH:mm') as time", "v"))

root
 |-- time: timestamp (nullable = true)
 |-- v: string (nullable = true)



,time,v
0,08:06,A
1,08:10,B
2,08:12,B
3,08:12,B
4,08:12,A
5,08:20,C
6,08:20,B
7,08:20,B


On considère la requête S2:

In [ ]:
S2 = (S1.groupBy( window(col("time"), "10 minute", "5 minute").alias("w"), "v")
      .agg(count("*").alias("nb"))
      .selectExpr("v","w.start as start", "w.end as end", "nb")
      .selectExpr("date_format(start, 'HH:mm') as start","v", "nb").orderBy("start", "v")
)

display(S2)

,start,v,nb
0,08:00,A,1
1,08:05,A,2
2,08:05,B,3
3,08:10,A,1
4,08:10,B,3
5,08:15,B,2
6,08:15,C,1
7,08:20,B,2
8,08:20,C,1


Question 1)
Proposer une solution pour calculer incrémentalement le résultat de la fenètre courante en réutilisant certains résultats des fenetres précédentes.
Pour cela définir par exemple une fonction traiterS2(donnees)

*On commence par faire des plages de taille décalage=5 puis on fera l'union des des tailles//decalage=10//5=2 plages adjacents.*

In [ ]:
from pyspark.sql.types import DateConverter
import math as m

def traiterS2(donnees, taille, decalage):
    # Calcul du début de la première fenêtre en arrondissant au multiple inférieur de 'taille'. (8h06 => début à 8h00)
    plage_min = donnees.collect()[0]['time']
    plage_min = plage_min.replace(minute=(plage_min.minute // taille) * taille)

    # Calcul de la fin de la dernière fenêtre en arrondissant au multiple supérieur de 'taille'. (8h20 => fin à 8h20)
    plage_max = donnees.collect()[-1]['time']
    plage_max = plage_max.replace(minute=((plage_max.minute + decalage) // taille) * taille)

    # Calcul du Plus Grand Commun Diviseur entre 'decalage' et 'taille' : permet d'obtenir la taille des fenetres
    # Les fenêtres doivent être d'une taille qui est un diviseur commun de 'decalage' et 'taille'.
    # Ceci est nécessaire pour s'assurer que les fenêtres puissent être concaténées ultérieurement
    # de manière à former des fenêtres de la taille spécifiée par 'taille'.
    pgcd = m.gcd(decalage, taille)

    # Calcul du nombre de fenêtres en fonction de 'decalage'.
    nb_fenetres = int((plage_max - plage_min).total_seconds() / 60) / pgcd

    # Génération de toutes les plages temporelles possibles.
    liste_plages = [plage_min + timedelta(minutes=pgcd * t) for t in range(int(nb_fenetres + 1))]

    # Initialisation d'un dictionnaire pour stocker les résultats par plage.
    dico = {plage: dict() for plage in liste_plages}

    # Boucle pour compter les occurrences de chaque valeur dans chaque plage.
    for time, v in donnees.collect():
        plage = time.replace(minute=(time.minute // pgcd ) * pgcd)
        if v in dico[plage]:
            dico[plage][v] += 1
        else:
            dico[plage][v] = 1

    # Calcul du nombre de plages à concaténer.
    nb_plage_concat = taille // pgcd

    # Boucle pour fusionner les données des plages adjacentes.
    res = dict()
    i=0
    while i < len(liste_plages):
        res_plage = dict()

        # Détermination du nombre de plages à joindre. (Pour les dernières , il est possible qu'il y ait moins de plages à concaténer)
        nb_joindre = nb_plage_concat if i < len(liste_plages) - nb_plage_concat + 1 else len(liste_plages) - i
        for j in range(nb_joindre):
            for v, nb in dico[liste_plages[i + j]].items():
                if v in res_plage:
                    res_plage[v] += dico[liste_plages[i + j]][v]
                else:
                    res_plage[v] = dico[liste_plages[i + j]][v]
        res[liste_plages[i]] = tuple(sorted(res_plage.items(), key=lambda x: x[0]))
        i+= decalage//pgcd
    return res

In [ ]:
print("Taille : 10, Décalage : 5")
res = traiterS2(S1, 10, 5)
df = pd.DataFrame([(date, date+timedelta(minutes=10), values) for date, values in res.items()], columns=['date_debut', 'date_fin', 'valeurs'])
df

Taille : 10, Décalage : 5


,date_debut,date_fin,valeurs
0,2023-11-10 08:00:00,2023-11-10 08:10:00,"((A, 1),)"
1,2023-11-10 08:05:00,2023-11-10 08:15:00,"((A, 2), (B, 3))"
2,2023-11-10 08:10:00,2023-11-10 08:20:00,"((A, 1), (B, 3))"
3,2023-11-10 08:15:00,2023-11-10 08:25:00,"((B, 2), (C, 1))"
4,2023-11-10 08:20:00,2023-11-10 08:30:00,"((B, 2), (C, 1))"


In [ ]:
# On donne un exemple où le décalage n'est pas un divisieur de la taille
print("Taille : 5, Décalage : 3")
res = traiterS2(S1, 5, 3)
df = pd.DataFrame([(date, date+timedelta(minutes=5), values) for date, values in res.items()], columns=['date_debut', 'date_fin', 'valeurs'])
df

Taille : 5, Décalage : 3


,date_debut,date_fin,valeurs
0,2023-11-10 08:05:00,2023-11-10 08:10:00,"((A, 1),)"
1,2023-11-10 08:08:00,2023-11-10 08:13:00,"((A, 1), (B, 3))"
2,2023-11-10 08:11:00,2023-11-10 08:16:00,"((A, 1), (B, 2))"
3,2023-11-10 08:14:00,2023-11-10 08:19:00,()
4,2023-11-10 08:17:00,2023-11-10 08:22:00,"((B, 2), (C, 1))"
5,2023-11-10 08:20:00,2023-11-10 08:25:00,"((B, 2), (C, 1))"


## Exercice 2 : Streaming

Proposer une implémentation pour l'exercice Streaming (celui de l'ER1 2022) vu en cours.

Cet exercice étant difficile à mettre en place avec sparkstreaming car l'arrivée des données est espacé de plus d'une heure, nous avons simulé la réflexion que nous pourrions avoir sur papier.

### Question 1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

# Définir le schéma du DataFrame
schema = StructType([
    StructField("evt", StringType(), True),
    StructField("heure", StringType(), True),
    StructField("categorie", StringType(), True),
    StructField("user", StringType(), True),
    StructField("points", IntegerType(), True)
])

# Données fournies
T1 = [
    ("e0", "07:30", "restau", "u1", 2),
    ("e1", "08:30", "musée", "u1", 5),
    ("e2", "09:30", "ciné", "u2", 3),
    ("e3", "09:50", "musée", "u3", 1),
    ("e4", "09:50", "ciné", "u2", 5)
]
T2 = [
    ("e5", "10:10", "ciné", "u5", 2),
    ("e6", "10:50", "restau", "u4", 4),
    ("e7", "10:40", "restau", "u5", 5),
    ("e8", "10:50", "musée", "u3", 4)]

T3 = [
    ("e9", "11:30", "musée", "u4", 3),
    ("e10", "12:05", "ciné", "u3", 2),
    ("e11", "11:10", "parc", "u1", 3)]

# Convertir les données en DataFrame avec l'heure en tant que StringType
df1 = spark.createDataFrame(T1, schema=schema)
df2 = spark.createDataFrame(T2, schema=schema)
df3 = spark.createDataFrame(T3, schema=schema)


def convert_to_timestamp(df):
    return df.withColumn("heure", to_timestamp(df["heure"], "HH:mm"))

# Appliquer la conversion en timestamp
df1 = convert_to_timestamp(df1)
df2 = convert_to_timestamp(df2)
df3 = convert_to_timestamp(df3)

# Afficher le DataFrame
df1.show()
df2.show()
df3.show()


+---+-------------------+---------+----+------+
|evt|              heure|categorie|user|points|
+---+-------------------+---------+----+------+
| e0|1970-01-01 07:30:00|   restau|  u1|     2|
| e1|1970-01-01 08:30:00|    musée|  u1|     5|
| e2|1970-01-01 09:30:00|     ciné|  u2|     3|
| e3|1970-01-01 09:50:00|    musée|  u3|     1|
| e4|1970-01-01 09:50:00|     ciné|  u2|     5|
+---+-------------------+---------+----+------+

+---+-------------------+---------+----+------+
|evt|              heure|categorie|user|points|
+---+-------------------+---------+----+------+
| e5|1970-01-01 10:10:00|     ciné|  u5|     2|
| e6|1970-01-01 10:50:00|   restau|  u4|     4|
| e7|1970-01-01 10:40:00|   restau|  u5|     5|
| e8|1970-01-01 10:50:00|    musée|  u3|     4|
+---+-------------------+---------+----+------+

+---+-------------------+---------+----+------+
|evt|              heure|categorie|user|points|
+---+-------------------+---------+----+------+
| e9|1970-01-01 11:30:00|    musée|  u

In [ ]:
def pseudo_stream(df, debut_str, debut_int, fin_str, fin_int):
  # window correspond à la plage horaire observée
  window = (col("heure") >= to_timestamp(lit(debut_str), "HH:mm")) & \
            (col("heure") < to_timestamp(lit(fin_str), "HH:mm"))
  df1_filtered = df.filter(window)

  # On calcule le nombre d'utilisateurs distincts et le nombre total de point par catégorie d'évènements
  resultat_t1 = df1_filtered.groupBy(
      col("categorie")
  ).agg(
      countDistinct(col("user")).alias("nbUser"),
      sum(col("points")).alias("totalPoints")
  ).withColumn("debut",lit(debut_int)).withColumn("fin", lit(fin_int)).select(
      col("debut"),
      col("fin"),
      col("categorie"),
      col("nbUser"),
      col("totalPoints")
  ).orderBy("categorie")
  return resultat_t1

In [ ]:
# au temps T1
res = pseudo_stream(df1, "07:00", 7, "10:00", 10)
res.show()

+-----+---+---------+------+-----------+
|debut|fin|categorie|nbUser|totalPoints|
+-----+---+---------+------+-----------+
|    7| 10|     ciné|     1|          8|
|    7| 10|    musée|     2|          6|
|    7| 10|   restau|     1|          2|
+-----+---+---------+------+-----------+



In [ ]:
# au temps T2
t2 = df1.union(df2)
res = pseudo_stream(t2, "08:00", 8, "11:00", 11)
res.show()

+-----+---+---------+------+-----------+
|debut|fin|categorie|nbUser|totalPoints|
+-----+---+---------+------+-----------+
|    8| 11|     ciné|     2|         10|
|    8| 11|    musée|     2|         10|
|    8| 11|   restau|     2|          9|
+-----+---+---------+------+-----------+



In [ ]:
# au temps T3
t3 = t2.union(df3)
res = pseudo_stream(t3, "09:00", 9, "12:00", 12)
res.show()

+-----+---+---------+------+-----------+
|debut|fin|categorie|nbUser|totalPoints|
+-----+---+---------+------+-----------+
|    9| 12|     ciné|     2|         10|
|    9| 12|    musée|     2|          8|
|    9| 12|     parc|     1|          3|
|    9| 12|   restau|     2|          9|
+-----+---+---------+------+-----------+



### question 2 :  gestion des données tardives

In [ ]:
# Données
T1 = [
    ("e0", "07:30", "restau", "u1", 2),
    ("e1", "08:30", "musée", "u1", 5),
    ("e2", "09:30", "ciné", "u2", 3),
    ("e3", "09:50", "musée", "u3", 1),
    ("e4", "09:50", "ciné", "u2", 5)
]
T2 = [
    ("e5", "07:10", "ciné", "u5", 2),
    ("e6", "10:50", "restau", "u4", 4),
    ("e7", "07:55", "restau", "u5", 5),
    ("e8", "10:50", "musée", "u3", 4)]

# Convertir les données en DataFrame avec l'heure en tant que StringType
df1 = spark.createDataFrame(T1, schema=schema)
df2 = spark.createDataFrame(T2, schema=schema)

def convert_to_timestamp(df):
    return df.withColumn("heure", to_timestamp(df["heure"], "HH:mm"))

# Appliquer la conversion
df1 = convert_to_timestamp(df1)
df2 = convert_to_timestamp(df2)

# Afficher le DataFrame
df1.show()
df2.show()

+---+-------------------+---------+----+------+
|evt|              heure|categorie|user|points|
+---+-------------------+---------+----+------+
| e0|1970-01-01 07:30:00|   restau|  u1|     2|
| e1|1970-01-01 08:30:00|    musée|  u1|     5|
| e2|1970-01-01 09:30:00|     ciné|  u2|     3|
| e3|1970-01-01 09:50:00|    musée|  u3|     1|
| e4|1970-01-01 09:50:00|     ciné|  u2|     5|
+---+-------------------+---------+----+------+

+---+-------------------+---------+----+------+
|evt|              heure|categorie|user|points|
+---+-------------------+---------+----+------+
| e5|1970-01-01 07:10:00|     ciné|  u5|     2|
| e6|1970-01-01 10:50:00|   restau|  u4|     4|
| e7|1970-01-01 07:55:00|   restau|  u5|     5|
| e8|1970-01-01 10:50:00|    musée|  u3|     4|
+---+-------------------+---------+----+------+



In [ ]:
def debut_max(df):
  return df.agg(max("heure").alias("max_heure")).collect()[0][0]

In [ ]:
# calcul de la borne inférieure de la plage de validite pour T1
borne_inf_validite = debut_max(df1) - timedelta(hours=2)
print(borne_inf_validite )

1970-01-01 07:50:00


In [ ]:
# ne conserver que le tuple de 7h50 mais pas celui de 7h10
df_filtered = df2.filter(col("heure") > borne_inf_validite)
df_filtered.show()

+---+-------------------+---------+----+------+
|evt|              heure|categorie|user|points|
+---+-------------------+---------+----+------+
| e6|1970-01-01 10:50:00|   restau|  u4|     4|
| e7|1970-01-01 07:55:00|   restau|  u5|     5|
| e8|1970-01-01 10:50:00|    musée|  u3|     4|
+---+-------------------+---------+----+------+



In [ ]:
# Résultat à T2
t2 = df1.union(df_filtered)

res1 = pseudo_stream(t2, "07:00", 7, "10:00", 10)
res2 = pseudo_stream(t2, "08:00", 8, "11:00", 11)
res = res1.union(res2)
res.show()

+-----+---+---------+------+-----------+
|debut|fin|categorie|nbUser|totalPoints|
+-----+---+---------+------+-----------+
|    7| 10|     ciné|     1|          8|
|    7| 10|    musée|     2|          6|
|    7| 10|   restau|     2|          7|
|    8| 11|     ciné|     1|          8|
|    8| 11|    musée|     2|         10|
|    8| 11|   restau|     1|          4|
+-----+---+---------+------+-----------+



### Question 3 Watermark

In [ ]:
# Definition du watermark
borne_inf_validite = debut_max(df1) - timedelta(hours=2)
print(borne_inf_validite)

1970-01-01 07:50:00


In [ ]:
# En mode append on n'affichera le résultat que lorsque l'on sera sûr que le resultat est complet
# ( borne inf de la plage de validité > borne sup de la fenetre)
if datetime.strptime("1970-01-01 07:00:00", "%Y-%m-%d %H:%M:%S") > borne_inf_validite + timedelta(hours=2) :
  t2 = df1.union(df_filtered)
  res1 = pseudo_stream(t2, "07:00", 7, "10:00", 10)
  res2 = pseudo_stream(t2, "08:00", 8, "11:00", 11)
  res = res1.union(res2)
  res.show()
else :
  df_filtered = t2.filter(col("heure")>borne_inf_validite + timedelta(hours=2))
  res1 = pseudo_stream(df_filtered, "07:00", 7, "10:00", 10)
  res1.show()

+-----+---+---------+------+-----------+
|debut|fin|categorie|nbUser|totalPoints|
+-----+---+---------+------+-----------+
+-----+---+---------+------+-----------+



## Exercice 3 : Mise en pratique Spark Streaming


In [ ]:
import os

dir_name = "/content/input"
os.makedirs(dir_name, exist_ok=True)

checkpoint = "/content/checkpoint"
os.makedirs(checkpoint, exist_ok=True)

streaming_query = None

Clean the input source

In [ ]:
# Vider l'input
!rm -f /content/input/*

np.random.seed(1)

NEXT_FILE_ID=1

def add_input_data(dir_name):
  global NEXT_FILE_ID
  nb_lines = 5
  with open(dir_name + f"/file_{NEXT_FILE_ID}.csv", "w") as f:
    content = [ f"{i} {a}\n" for i,a in enumerate(np.random.randint(1, 20, nb_lines))]
    for l in content:
      print(l)
    f.writelines(''.join(content))
    print("added file:", NEXT_FILE_ID)
    NEXT_FILE_ID += 1

print("function defined")

function defined


Define the input stream

In [ ]:
schema = "ID int, note int"
input_stream = spark.readStream.schema(schema).option("delimiter", " ").option("maxFilesPerTrigger", 1).csv(dir_name)

Define a **query** on the input stream

In [ ]:
query = input_stream.groupBy("ID").agg(avg("note").alias("moyenne")).orderBy("ID")
# print("Streaming is: ", query.isStreaming)

Define the **output** stream and **execute the streaming query**

In [ ]:
if(streaming_query is not None and streaming_query.isActive):
  print("Stopping current streaming query before defining a new one.")
  streaming_query.stop()

streaming_query = query.writeStream.format("memory").queryName("Result").outputMode("complete").start()

print("Active stream:", streaming_query.isActive)

Active stream: True


Produire des données d'entrée

In [ ]:
add_input_data(dir_name)

0 6

1 12

2 13

3 9

4 10

added file: 1


Afficher le résultat de la requete de streaming

In [ ]:
%%sql
select * from Result

,ID,moyenne


In [ ]:
add_input_data(dir_name)

0 12

1 6

2 16

3 1

4 17

added file: 2


In [ ]:
%%sql
select * from Result

,ID,moyenne


In [ ]:
# streaming_query.lastProgress

### Question 1 : Watermark
Adapter l'exemple ci dessus et proposer une requete avec un regroupement par fenetre et une watermark

In [ ]:
dir_name = "/content/input2"
os.makedirs(dir_name, exist_ok=True)

checkpoint = "/content/checkpoint2"
os.makedirs(checkpoint, exist_ok=True)

streaming_query = None

In [ ]:
#  Nous avons overwrite la fonction afin d'ajouter un timestamp de date de création et un séparateur par ";"
from datetime import datetime
from pyspark.sql.functions import expr
import time

def add_input_data(dir_name):
    global NEXT_FILE_ID
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(f"{dir_name}/file_{NEXT_FILE_ID}.csv", "w") as f:
        content = [f"{timestamp};{i};{a}\n" for i, a in enumerate(np.random.randint(1, 20, 5))]
        for l in content:
            print(l)
        f.writelines(''.join(content))
        print("added file:", NEXT_FILE_ID)
        NEXT_FILE_ID += 1

In [ ]:
schema = "timestamp timestamp, ID int, note int"

input_stream = spark.readStream.schema(schema).option("delimiter", ";").option("maxFilesPerTrigger", 1).csv(dir_name)

query = (
    input_stream
    .withWatermark("timestamp", "5 seconds")
    .groupBy(window("timestamp", "5 seconds", "5 seconds"), "ID")
    .agg(avg("note").alias("moyenne"))
    .orderBy("window", "ID")
)

print("Streaming is: ", query.isStreaming)

Streaming is:  True


In [ ]:
if streaming_query is not None and streaming_query.isActive:
    print("Stopping current streaming query before defining a new one.")
    streaming_query.stop()

streaming_query = query.writeStream.format("memory").queryName("Result2").outputMode("complete").start()

print("Active stream:", streaming_query.isActive)

Active stream: True


In [ ]:
add_input_data(dir_name)

2023-11-17 17:23:54;0;2

2023-11-17 17:23:54;1;13

2023-11-17 17:23:54;2;8

2023-11-17 17:23:54;3;14

2023-11-17 17:23:54;4;7

added file: 3


In [ ]:
%%sql
select * from Result2
order by window desc

,window,ID,moyenne


### Question 2: Jointure
Proposer une requete avec jointure entre un flux et une table

In [ ]:
#On overwrite la fonction pour coller à la problématique.
!rm -f /content/input3/*

np.random.seed(1)

NEXT_FILE_ID=1

import numpy as np
import csv

NEXT_FILE_ID = 1

def add_input_data(dir_name):
    global NEXT_FILE_ID
    nb_lines = 5
    with open(dir_name + f"/file_{NEXT_FILE_ID}.csv", "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["ID1", "value1"])
        content = [(i, a) for i, a in enumerate(np.random.randint(1, 20, nb_lines))]
        writer.writerows(content)
        print("added file:", NEXT_FILE_ID)
        NEXT_FILE_ID += 1

print("function defined")

function defined


In [ ]:
from pyspark.sql.functions import avg
streaming_query =None
# Initialisation de Spark
spark = SparkSession.builder.appName("streaming_join_example").getOrCreate()

# créer directory du stream et son checkpointing
dir_name = "/content/input3"
os.makedirs(dir_name, exist_ok=True)
checkpoint = "/content/checkpoint3"
os.makedirs(checkpoint, exist_ok=True)

#crer streaming
add_input_data(dir_name)
schema = "ID1 int, value1 int"
streaming_df = spark.readStream.schema(schema).csv(dir_name)

# créer dataframe
data_table = [(i,int(a)) for _ in range(1) for i,a in enumerate(np.random.randint(1, 20, 5)) ]
static_schema = StructType([
    StructField("ID2", IntegerType()),
    StructField("value2", IntegerType())])
static_table = spark.createDataFrame(data_table, static_schema)

# jointure
joined_df = streaming_df.join(static_table, streaming_df.ID1 == static_table.ID2)\
.select(streaming_df.ID1.alias("ID"), streaming_df.value1, static_table.value2, ((streaming_df.value1 + static_table.value2)/2).alias("moy"))\
.groupBy("ID").agg(avg(col("moy")).alias("moyenne")).select("ID","moyenne")

added file: 1


In [ ]:
if(streaming_query is not None and streaming_query.isActive):
  print("Stopping current streaming query before defining a new one.")
  streaming_query.stop()

streaming_query = joined_df.writeStream.format("memory").outputMode("complete").queryName("Result_join1").start()

print("Active stream:", streaming_query.isActive)

Active stream: True


In [ ]:
add_input_data(dir_name)

added file: 2


In [ ]:
# si le tableau est vide, patienter et relancer la requete
result_df = spark.sql("SELECT ID, moyenne, dense_rank() over(order by moyenne desc) as rang FROM Result_join1 ORDER BY rang")
result_df.show()

+---+-------+----+
| ID|moyenne|rang|
+---+-------+----+
|  2|  13.25|   1|
|  4|  12.75|   2|
|  1|   9.25|   3|
|  0|    8.0|   4|
|  3|   6.25|   5|
+---+-------+----+

